<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%AA%D8%B9%D8%AF%D9%8A%D9%84%D8%A7%D8%AA_%D8%B9_%D8%A7%D9%84%D8%B1%D8%B3%D9%88%D9%85%D8%A7%D8%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt

def plot_bollinger_bands(data, window=20, num_std_dev=2):
    """رسم بولنجر باندز على مخطط بياني."""
    # حساب المتوسط المتحرك والانحراف المعياري
    data['Middle_Band'] = data['close'].rolling(window=window).mean()
    data['Std_Dev'] = data['close'].rolling(window=window).std()

    # حساب الحد العلوي والحد السفلي
    data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
    data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)

    # طباعة القيم
    print("Middle Band:", data['Middle_Band'].iloc[-1])
    print("Upper Band:", data['Upper_Band'].iloc[-1])
    print("Lower Band:", data['Lower_Band'].iloc[-1])

    # رسم المخطط
    plt.figure(figsize=(12, 6))
    plt.plot(data['close'], label='Close Price', color='blue')
    plt.plot(data['Middle_Band'], label='Middle Band', color='orange')
    plt.plot(data['Upper_Band'], label='Upper Band', color='green')
    plt.plot(data['Lower_Band'], label='Lower Band', color='red')
    plt.fill_between(data.index, data['Upper_Band'], data['Lower_Band'], color='lightgray', alpha=0.5)
    plt.title('Bollinger Bands')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

# مثال على كيفية استخدام الدالة
# تأكد من أن لديك بيانات في DataFrame باسم `data` تحتوي على عمود 'close'
# plot_bollinger_bands(data)


In [ ]:
import matplotlib.pyplot as plt

def plot_premium_discount(data):
    """رسم Premium وDiscount على مخطط بياني."""
    # تأكد من أن البيانات تحتوي على الأعمدة المطلوبة
    data['Premium'] = data['close'] - data['low'].rolling(window=20).min()
    data['Discount'] = data['high'].rolling(window=20).max() - data['close']

    # طباعة القيم الأخيرة
    print("Last Premium Value:", data['Premium'].iloc[-1])
    print("Last Discount Value:", data['Discount'].iloc[-1])

    # رسم المخطط
    plt.figure(figsize=(12, 6))
    plt.plot(data.index, data['Premium'], label='Premium', color='blue')
    plt.plot(data.index, data['Discount'], label='Discount', color='red')
    plt.fill_between(data.index, data['Premium'], color='lightblue', alpha=0.5)
    plt.fill_between(data.index, data['Discount'], color='lightcoral', alpha=0.5)
    plt.title('Premium and Discount Bands')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

# مثال على كيفية استخدام الدالة
# تأكد من أن لديك بيانات في DataFrame باسم `data` تحتوي على الأعمدة 'close', 'low', 'high'
# plot_premium_discount(data)


In [ ]:
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """حساب مؤشر MACD"""
    data['EMA_short'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

def plot_macd(data):
    """رسم مؤشر MACD."""
    plt.figure(figsize=(12, 6))
    plt.plot(data.index, data['MACD'], label='MACD', color='blue')
    plt.plot(data.index, data['Signal_Line'], label='Signal Line', color='orange')
    plt.axhline(0, color='black', lw=0.5, ls='--')
    plt.title('MACD Indicator')
    plt.xlabel('Date')
    plt.ylabel('MACD Value')
    plt.legend()
    plt.grid()
    plt.show()

# في دالة build_strategy
data = calculate_macd(data)
plot_macd(data)


In [ ]:
import matplotlib.pyplot as plt

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks وإعادتها للرسم."""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i - 1] and data['close'].iloc[i - 1] < data['close'].iloc[i - 2]:
                bullish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))
            elif data['close'].iloc[i] < data['close'].iloc[i - 1] and data['close'].iloc[i - 1] > data['close'].iloc[i - 2]:
                bearish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")

    return bullish_blocks, bearish_blocks


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        # (تأكد من إضافة جميع العمليات السابقة هنا)

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)

        # حساب مناطق Order Blocks
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # رسم البيانات
        plt.figure(figsize=(14, 7))
        plt.plot(data['close'], label='Close Price', color='blue')

        # رسم مناطق طلبات الشراء
        for block in bullish_blocks:
            plt.axhline(y=block[1], color='green', linestyle='--', label='Bullish Block' if block == bullish_blocks[0] else "")

        # رسم مناطق طلبات البيع
        for block in bearish_blocks:
            plt.axhline(y=block[1], color='red', linestyle='--', label='Bearish Block' if block == bearish_blocks[0] else "")

        plt.title('Order Blocks')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

        # (استمر في تنفيذ باقي الشروط لفتح الأوامر)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i - 1] and data['close'].iloc[i - 1] < data['close'].iloc[i - 2]:
                bullish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))
            elif data['close'].iloc[i] < data['close'].iloc[i - 1] and data['close'].iloc[i - 1] > data['close'].iloc[i - 2]:
                bearish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))

        # طباعة مناطق الطلبات
        print("Bullish Order Blocks:")
        for block in bullish_blocks:
            print(f"Time: {block[0]}, Price: {block[1]}")

        print("Bearish Order Blocks:")
        for block in bearish_blocks:
            print(f"Time: {block[0]}, Price: {block[1]}")

    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")

    return bullish_blocks, bearish_blocks


In [ ]:
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i - 1] and data['close'].iloc[i - 1] < data['close'].iloc[i - 2]:
                bullish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))
            elif data['close'].iloc[i] < data['close'].iloc[i - 1] and data['close'].iloc[i - 1] > data['close'].iloc[i - 2]:
                bearish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))

        # طباعة آخر نقطتين من مناطق طلبات الشراء والبيع
        if bullish_blocks:
            print("آخر نقطتين من مناطق طلبات الشراء:")
            print(bullish_blocks[-2:])  # طباعة آخر نقطتين
        else:
            print("لا توجد مناطق طلبات شراء.")

        if bearish_blocks:
            print("آخر نقطتين من مناطق طلبات البيع:")
            print(bearish_blocks[-2:])  # طباعة آخر نقطتين
        else:
            print("لا توجد مناطق طلبات بيع.")

    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")

    return bullish_blocks, bearish_blocks


In [ ]:
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks لأحدث نقطة."""
    bullish_blocks = []
    bearish_blocks = []

    try:
        # تحقق من وجود بيانات كافية
        if len(data) < 3:
            print("لا توجد بيانات كافية لحساب مناطق الطلبات.")
            return bullish_blocks, bearish_blocks

        # تحليل آخر ثلاث نقاط فقط
        for i in range(len(data) - 2, len(data)):
            if data['close'].iloc[i] < data['close'].iloc[i - 1] and data['close'].iloc[i - 1] > data['close'].iloc[i - 2]:
                bullish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))
            elif data['close'].iloc[i] > data['close'].iloc[i - 1] and data['close'].iloc[i - 1] < data['close'].iloc[i - 2]:
                bearish_blocks.append((data.index[i - 1], data['close'].iloc[i - 1]))

        # طباعة المناطق المحسوبة
        print(f"مناطق طلبات الشراء (Bullish Blocks): {bullish_blocks}")
        print(f"مناطق طلبات البيع (Bearish Blocks): {bearish_blocks}")

    except Exception as e:
        print(f"حدث خطأ أثناء حساب مناطق الطلبات: {e}")

    return bullish_blocks, bearish_blocks
